In [1]:
import os
import json
import pandas as pd

## Load Fantom transactions

The first stage of this snapshot used an archive of the Fantom DAG, which was queried from public RPC servers.
The Fantom archive contains blocks ranging from 54647659 to 65653613 (2023-01-29 to 2023-07-12).

In [2]:
fantom_txs_filename = os.path.expanduser("../var/fantom-txs.csv.gz")
txs = pd.read_csv(fantom_txs_filename)
txs.head()

,block_id,tx_hash,method,from,to,quantity
0,54647659,0x68b471267b020fb7be707817bbe822da9a18f45071c7...,0x627dd56a,0x5cD5AFF4A3b42b59651BAB5Eb45c07375f60e905,0x3319161b131401124E61ffCDFf0aa7f6FDcbbFEd,0
1,54647659,0xa873edd8d865c95afd33d5ebeb34cad3df4843b2c769...,0x16711fdc,0x53e0466D46306e9824590988f50B0EEAcA1A6Dae,0x6Ff283ac7BB7F66475Ca79aA549c6f689cDB89Dd,0
2,54647659,0x68f2f5c608c220d3ff0c421eed9812b7e9d6124d4adb...,0xac882d06,0x656DAA21A427a1A290dD78AC581fFe8CbfF16066,0x6b120e310A892D1Cbe0F89E76Bf7Cb0da3a85BCe,0
3,54647659,0xd1e53d2e2bcf779b4061a4dc7080a33d4f270b72c61a...,0x627dd56a,0x01883C8E7Abb7C2Ef03d802F77AA567a254e17d7,0x0000900e00070d8090169000D2B090B67f0c1050,0
4,54647659,0x961463fa4ffc36d9d1fea04060b9c57072950a81a5ca...,0x58b58d44,0x211cB8e0740AdF9066453e627dcc196225c8F0E7,0xe0df6a0840C4f31E81C3d4B11Ba8B579F2ba8b1a,0


## Identify interacting wallet addresses

This snapshot begins by identifying all the addresses that ever interacted with Morphex contracts.
This list of addresses will be used in the next stage to query balances on-chain.

In [3]:
mpx_erc20_address = "0x66eEd5FF1701E6ed8470DC391F05e27B1d0657eb"
mlp_erc20_address = "0xd5c313DE2d33bf36014e6c659F13acE112B80a8E"
vault_address = "0x3CB54f0eB62C371065D739A34a775CC16f46563e"
mlp_manager_address = "0xA3Ea99f8aE06bA0d9A6Cf7618d06AEa4564340E9"
musd_erc20_address = "0xB7209EbCBF71c0ffA1585B4468A11CFfdcDBB9a9"
esmpx_erc20_address = "0xe0f606e6730bE531EeAf42348dE43C2feeD43505"
reward_router_address = "0x20De7f8283D377fA84575A26c9D484Ee40f55877"
reward_reader_address = "0x512F8D4E28EB53A6d036aEDA9C5a4D1De6DBD543"
weth_erc20_address = "0x21be370D5312f44cB42ce377BC9b8a0cEF1A4C83"
tracker_staked_mlp_address = "0xa4157E273D88ff16B3d8Df68894e1fd809DbC007" # staked MPX
tracker_staked_mpx_address = "0x49A97680938B4F1f73816d1B70C3Ab801FAd124B" # fee + staked MLP
equalizer_wftm_mpx_address = "0xdE26e98d868FE02fFfb6DF26E638995124d3Ca13"
equalizer_wftm_mpx_address_new = "0xF8eed2665FD11a8431fc41b2582fD5E72a1606f0"
equalizer_gauge_1 = "0x27F7cf5e918311AAF5E7185b5BcDAc158dFacf53" # 
equalizer_gauge_2 = "0x7778a0B4688321c4E705d4e9F1A072f6F1579Bf8" # equalizer

### MPX ERC-20

In [4]:
mpx_txs = txs.query(f'to == "{mpx_erc20_address}"')
len(mpx_txs)

10879

In [5]:
unique_mpx_addresses = set(mpx_txs['from'])
len(unique_mpx_addresses)

4233

### Reward Router

In [6]:
reward_router_txs = txs.query(f'to == "{reward_router_address}"')
len(reward_router_txs)

46739

In [7]:
unique_reward_addresses = set(reward_router_txs['from'])
len(unique_reward_addresses)

1585

### Equalizer WFTM-MPX LP

In [8]:
equalizer_txs = txs.query(f'to == "{equalizer_wftm_mpx_address}"')
len(equalizer_txs)

2503

In [9]:
unique_equalizer_addresses = set(equalizer_txs['from'])
len(unique_equalizer_addresses)

572

### Equalizer WFTM-MPX Gauge

In [10]:
equalizer_gauge_txs = txs.query(f'to == "{equalizer_gauge_1}"')
len(equalizer_gauge_txs)

3838

In [11]:
unique_equalizer_gauge_addresses = set(equalizer_gauge_txs['from'])
len(unique_equalizer_gauge_addresses)

265

### Combine unique addresses

In [12]:
addresses = unique_reward_addresses | unique_mpx_addresses | unique_equalizer_addresses | unique_equalizer_gauge_addresses
addresses = list(addresses)

# get balances for Equalizer LPs
addresses.append(equalizer_wftm_mpx_address)
addresses.append(equalizer_wftm_mpx_address_new) 
# also get balances for Equalizer Gauges
addresses.append(equalizer_gauge_1)
addresses.append(equalizer_gauge_2)

with open("../data/fantom-addresses.json", "w") as f:
    json.dump(addresses, f)
len(addresses)

4509

## Call contracts on-chain to obtain balances

Fantom RPCs providing full/archival operation permit `eth_call` to be executed against a historical block.
For this snapshot, we queried contracts on-chain at block `64884840` (June 30, 2:20 UTC).

In [13]:
def parse_address(item):
    return item.split(",")[2].split("'")[1]

### MPX ERC-20

Obtain balances for all interacting addresses, including Equalizer LPs.
MPX that is single-staked will be counted later.

In [14]:
with open("../data/fantom-mpx-results.json", "r") as f:
    mpx_erc20_call_results = json.load(f)
len(mpx_erc20_call_results)

4509

In [15]:
mpx_erc20_balances = {}
for key, value in mpx_erc20_call_results.items():
    if value > 0:
        mpx_erc20_balances[parse_address(key)] = value

df = pd.DataFrame.from_dict(mpx_erc20_balances, orient="index", columns=["balance"])
df["address"] = df.index
df.reset_index(drop=True, inplace=True)
df = df[["address", "balance"]]
df['balance'] = df['balance'].apply(lambda x: int(x)).values.tolist()
df.to_csv("../products/fantom-mpx-balances.csv", index=False, float_format='%g')
sum(df['balance']) / 1e18

1614428.0117833277

### Single-staked MPX

In [16]:
with open("../data/fantom-staked-mpx-results.json", "r") as f:
    mpx_call_results = json.load(f)
len(mpx_call_results)

4509

In [17]:
mpx_balances = {}
for key, value in mpx_call_results.items():
    if value > 0:
        mpx_balances[parse_address(key)] = value

df = pd.DataFrame.from_dict(mpx_balances, orient="index", columns=["balance"])
df["address"] = df.index
df.reset_index(drop=True, inplace=True)
df = df[["address", "balance"]]
df['balance'] = df['balance'].apply(lambda x: int(x)).values.tolist()
df.to_csv("../products/fantom-smpx-balances.csv", index=False, float_format='%g')
sum(df['balance']) / 1e18

7574403.20438577

### Staked MLP

In [18]:
with open("../data/fantom-staked-mlp-results.json", "r") as f:
    mlp_call_results = json.load(f)
len(mlp_call_results)

4509

In [19]:
mlp_balances = {}
for key, value in mlp_call_results.items():
    if value > 0:
        mlp_balances[parse_address(key)] = value

df = pd.DataFrame.from_dict(mlp_balances, orient="index", columns=["balance"])
df["address"] = df.index
df.reset_index(drop=True, inplace=True)
df = df[["address", "balance"]]
df['balance'] = df['balance'].apply(lambda x: int(x)).values.tolist()
df.to_csv("../products/fantom-fsmlp-balances.csv", index=False, float_format='%g')
sum(df['balance']) / 1e18

1461026.6865648555

### Equalizer WFTM-MPX LP

First we obtain the MPX balance for the Equalizer LP.
These balances are queried from the MPX ERC-20 contract.

In [20]:
pool_mpx_total = mpx_erc20_balances[equalizer_wftm_mpx_address]
pool_mpx_total / 1e18

1281894.1184612277

Then we obtain the balances of the Equalizer WFTM-MPX LP.
This represents the distribution of MPX provided for liquidity.

In [21]:
with open("../data/fantom-equalizer-wftm-mpx-results.json", "r") as f:
    equalizer_call_results = json.load(f)
len(equalizer_call_results)

4509

In [33]:
equalizer_lp_balances = {}
for key, value in equalizer_call_results.items():
    if value > 0:
        equalizer_lp_balances[parse_address(key)] = value

pool_lp_total = sum(equalizer_lp_balances.values())
pool_lp_total / 1e18

851396.9011039473

### Equalizer WFTM-MPX Gauges

Most of the LP is controlled by a gauge.
So, we must examine gauge contract balances to find the actual addresses that control the MPX.

In [24]:
with open("../data/fantom-equalizer-gauge-1-results.json", "r") as f:
    equalizer_gauge_1_call_results = json.load(f)
len(equalizer_gauge_1_call_results)

4509

In [25]:
equalizer_gauge_1_balances = {}
for key, value in equalizer_gauge_1_call_results.items():
    if value > 0:
        equalizer_gauge_1_balances[parse_address(key)] = value

gauge_1_lp_total = sum(equalizer_gauge_1_balances.values())
sum(equalizer_gauge_1_balances.values()) / 1e18

524974.0046707705

In [26]:
with open("../data/fantom-equalizer-gauge-2-results.json", "r") as f:
    equalizer_gauge_2_call_results = json.load(f)
len(equalizer_gauge_2_call_results)

4509

In [27]:
equalizer_gauge_2_balances = {}
for key, value in equalizer_gauge_2_call_results.items():
    if value > 0:
        equalizer_gauge_2_balances[parse_address(key)] = value

gauge_2_lp_total = sum(equalizer_gauge_2_balances.values())
sum(equalizer_gauge_2_balances.values()) / 1e18

11149.893598054376

### Attribute MPX to LP and Gauge

Finally, we attribute MPX to original addresses.
The MPX controlled by each address is held in proportion to the LP tokens controlled by that address.

Step 1. Examine the LP and directly assign LP to non-gauge addresses.

In [36]:
equalizer_lp_attributed_balances = {}
for key, value in equalizer_call_results.items():
    if value > 0 and parse_address(key) not in [equalizer_gauge_1, equalizer_gauge_2]:
        equalizer_lp_attributed_balances[parse_address(key)] = value
sum(equalizer_lp_attributed_balances.values()) / 1e18

1123.1288125757815

Step 2. Examine Gauges and assign LP to addresses.
First determine how much LP is controlled by a gauge, then assign LP based on proportion of gauge.

In [38]:
lp_controlled_by_gauge_1 = equalizer_lp_balances[equalizer_gauge_1]
lp_controlled_by_gauge_1 / 1e18

839123.0124607284

In [39]:
for key, value in equalizer_gauge_1_call_results.items():
    if value > 0 and parse_address(key) not in [equalizer_gauge_1, equalizer_gauge_2]:
        equalizer_lp_attributed_balances[parse_address(key)] = (value / gauge_1_lp_total) * lp_controlled_by_gauge_1
sum(equalizer_lp_attributed_balances.values()) / 1e18

840188.8691212935

In [40]:
lp_controlled_by_gauge_2 = equalizer_lp_balances[equalizer_gauge_2]
lp_controlled_by_gauge_2 / 1e18

11150.759830643088

In [41]:
for key, value in equalizer_gauge_2_call_results.items():
    if value > 0 and parse_address(key) not in [equalizer_gauge_1, equalizer_gauge_2]:
        equalizer_lp_attributed_balances[parse_address(key)] = (value / gauge_2_lp_total) * lp_controlled_by_gauge_2
sum(equalizer_lp_attributed_balances.values()) / 1e18

848706.3446693355

In [42]:
(1 - ((pool_lp_total - sum(equalizer_lp_balances.values())) / pool_lp_total)) * 100

100.0

We have attributed all of the LP.

In [46]:
df = pd.DataFrame.from_dict(equalizer_lp_attributed_balances, orient="index", columns=["lp_balance"])
df["address"] = df.index
df.reset_index(drop=True, inplace=True)
df = df[["address", "lp_balance"]]
df['mpx_balance'] = (df['lp_balance'] / pool_lp_total) * pool_mpx_total
df['lp_balance'] = df['lp_balance'].apply(lambda x: int(x)).values.tolist()
df['mpx_balance'] = df['mpx_balance'].apply(lambda x: int(x)).values.tolist()
df.to_csv("../products/fantom-equalizer-balances.csv", index=False, float_format='%g')
sum(df['lp_balance']) / 1e18

848706.3446693354

In [47]:
sum(df['mpx_balance']) / 1e18

1277843.1189045638